In [1]:
using GuidedProposals, DiffusionDefinition, ObservationSchemes
const GP = GuidedProposals
const DD = DiffusionDefinition
const OBS = ObservationSchemes

using StaticArrays, Random, Plots, LinearAlgebra
using Statistics

# seed used for this tutorial
Random.seed!(100)

## data generation ##
@diffusion_process FavettoSamsonModified{T,K} begin
    :dimensions
    process --> 2
    wiener --> 2

    :parameters
    (α, β, λ, μ, σ1, σ2) --> T
    dose --> K

    :additional
    diagdiff --> true
    constdiff --> true
end

DiffusionDefinition.b(t, x, P::FavettoSamsonModified) = @SVector [
    P.α * P.dose(t) - (P.λ +P.β)*x[1] + (P.μ-P.λ)*x[2],
    P.λ*x[1] - (P.μ-P.λ)*x[2]
]

DiffusionDefinition.σ(t, x, P::FavettoSamsonModified) = SDiagonal(P.σ1, P.σ2)

A new struct `FavettoSamsonModified{T, K}` has been defined.


In [2]:
@diffusion_process FavettoSamsonAux{K,Q,R} begin
    :dimensions
    process --> 2
    wiener --> 2

    :parameters
    (α, β, λ, μ, σ1, σ2) --> K
    dose --> Q
    :auxiliary_info
    t0 --> Float64
    T --> Float64
    vT --> R

    :additional
    constdiff --> true
    linear --> true
end

DiffusionDefinition.B(t, P::FavettoSamsonAux) = @SMatrix [-(P.λ+ P.β) P.β; P.λ -P.μ]
DiffusionDefinition.β(t, P::FavettoSamsonAux) = @SVector [P.α* P.dose(t), 0.0]
DiffusionDefinition.σ(t, P::FavettoSamsonAux) = SDiagonal(P.σ1, P.σ2)

@load_diffusion FavettoSamsonAux

A new struct `FavettoSamsonAux{K, Q, R}` has been defined.


In [3]:
function loglikelihood(true_data, curr_data, σ)
    ll = (norm(curr_data-true_data, 2) / σ)^2
    return ll
end

loglikelihood (generic function with 1 method)

In [4]:
# define a transition kernel for θ
function customkernel(θ, s::Symbol, scale=0.1)
	θ° = deepcopy(θ)
	θ°[s] += 2.0*scale*(rand()-0.5)
	θ°
end

customkernel (generic function with 2 methods)

In [5]:
function update_random_walker(q, a_r, n) # transition kernel q + acceptance rate + number of batch of 100 iteratiions: n
    q_move = min(0.01,1/sqrt(n))
    if a_r > 0.44
        q = q * exp(q_move)
    elseif a_r < 0.44
        q = q / exp(q_move)
    else
        q = q
    end
    q
end

update_random_walker (generic function with 1 method)

In [6]:
function mcmc_MH(AuxLaw, recording, dt, θ, s::Symbol; q=0.3, n)
    tts = OBS.setup_time_grids(recording, dt)
    PP = build_guid_prop(AuxLaw, recording, tts)
    PP° = deepcopy(PP)
    y1 = rand(recording.x0_prior)
    XX, WW, Wnr = rand(PP, y1)
    XX°, WW° = trajectory(PP)
    ll = loglikhd(PP, XX)
#     ll = 0
    ll_list = Float64[ll,]
    acpt_r = 0
    for i in 1:n
        θ° = customkernel(θ, s, q)
        DD.set_parameters!(PP°, θ°)
        recompute_guiding_term!(PP°)
        _, ll° = GP.solve_and_ll!(XX°, WW, PP°, y1)
#         P = FavettoSamsonModified(θ..., t->t/(1+t^2))
#         X = rand(P, tt, y1)
#         data_new = X.x
#         ll_new = loglikelihood(data_new, data, 1.0)
        
        if rand() < exp(ll°-ll)
            XX, PP, θ, XX°, PP°, θ° = XX°, PP°, θ°, XX, PP, θ
            ll = ll°
            append!(ll_list, ll)
            acpt_r += 1
        end
        if i % 100 == 0
            q = update_random_walker(q, acpt_r / 100, i)
            acpt_r = 0
        end
    end
    return ll_list
end

mcmc_MH (generic function with 1 method)

In [7]:
function credible_interval(data::Vector{Float64}, q::Array{Float64,1}=[0.025,0.975])
    a, b = quantile(LL_list,[0.025,0.975])
    println("likelihood based credible interval is [$a,$b]")
end

credible_interval (generic function with 2 methods)

In [8]:
θ = [116.7, 5.83, 1.25, 2.25, 2.0, 1.0]
P = FavettoSamsonModified(θ..., t->t/(1+t^2))
tt, y1 = 0.0:0.01:10.0, @SVector [0.0, 0.0]
X = rand(P, tt, y1)
data = map(
    x->(x[1], x[2] + randn()),
    collect(zip(X.t, X.x))
)[2:end]

data1 = map(
    x->(x[1], x[2][1] + x[2][2] + 1randn()),
    collect(zip(X.t, X.x))
)[2:end]

1000-element Vector{Tuple{Float64, Float64}}:
 (0.01, -0.10488417914007307)
 (0.02, 1.1551022158307789)
 (0.03, 3.4776852092022437)
 (0.04, -1.0335863226159137)
 (0.05, 0.509378057578065)
 (0.06, 0.8893048941575232)
 (0.07, 1.0315304087120607)
 (0.08, 1.169007732988062)
 (0.09, 0.6367200652969395)
 (0.1, 2.3487506520709425)
 (0.11, 2.8868413166934923)
 (0.12, 0.6685048083576012)
 (0.13, 2.320568262705999)
 ⋮
 (9.89, 3.2714970288664196)
 (9.9, 2.7952338547975595)
 (9.91, 3.559453617231237)
 (9.92, 4.585819272247674)
 (9.93, 3.8666658060895376)
 (9.94, 5.156077760461788)
 (9.95, 5.1155874941154265)
 (9.96, 4.126804257445513)
 (9.97, 2.5688380178390258)
 (9.98, 3.7219872154446763)
 (9.99, 4.396128972988999)
 (10.0, 5.079784562868041)

In [9]:
recording = (
    #P = FavettoSamsonModified([10, 5, #==# 0.0 #==#, 2, 1, 1]..., t-> 1/(1+t^2)),
    P = FavettoSamsonModified([116.7, 5.83, 1.25, 2.25, 2.0, #==# 0.0 #==#]..., t->t/(1+t^2)), # diffusion law
    obs = load_data(
        ObsScheme(
            LinearGsnObs(
                0.0, (@SVector [0.0]); # dummy variables indicating datatype of observations
                L=(@SMatrix [1.0 1.0]), # observation operator
                Σ=(@SMatrix [0.1]) #1e-2*SDiagonal{2,Float64}(I) # noise on the observations
            )
        ),
        data1
    ),
    t0 = 0.0,
    x0_prior = KnownStartingPt(y1),
)

(P = FavettoSamsonModified{Float64, var"#8#9"}(116.7, 5.83, 1.25, 2.25, 2.0, 0.0, var"#8#9"()), obs = LinearGsnObs{0, 1, SVector{1, Float64}, NoFirstPassageTimes, SMatrix{1, 2, Float64, 2}, SVector{1, Float64}, SMatrix{1, 1, Float64, 1}, Any}[LinearGsnObs{0, 1, SVector{1, Float64}, NoFirstPassageTimes, SMatrix{1, 2, Float64, 2}, SVector{1, Float64}, SMatrix{1, 1, Float64, 1}, Any}([1.0 1.0], [0.0], [0.1], [-0.10488417914007307], 0.01, false, Any[]), LinearGsnObs{0, 1, SVector{1, Float64}, NoFirstPassageTimes, SMatrix{1, 2, Float64, 2}, SVector{1, Float64}, SMatrix{1, 1, Float64, 1}, Any}([1.0 1.0], [0.0], [0.1], [1.1551022158307789], 0.02, false, Any[]), LinearGsnObs{0, 1, SVector{1, Float64}, NoFirstPassageTimes, SMatrix{1, 2, Float64, 2}, SVector{1, Float64}, SMatrix{1, 1, Float64, 1}, Any}([1.0 1.0], [0.0], [0.1], [3.4776852092022437], 0.03, false, Any[]), LinearGsnObs{0, 1, SVector{1, Float64}, NoFirstPassageTimes, SMatrix{1, 2, Float64, 2}, SVector{1, Float64}, SMatrix{1, 1, Float

In [10]:
DD.const_parameter_names(::Type{<:FavettoSamsonModified}) = (:α, :β, :λ,:μ,:σ1) 
DD.const_parameter_names(::Type{<:FavettoSamsonAux}) = (:α,:β,:λ, :μ, :σ1, :t0, :T, :vT, :xT)
LL_list = mcmc_MH(FavettoSamsonAux, recording, 0.01, Dict(:σ2=> 0.0), :σ2; q = 0.3, n = 10^4)
credible_interval(LL_list)

likelihood based credible interval is [-1783.8172740560738,-1780.7286548301179]
